# Generierung von Gegenreden zu Verschwörungstheorien mit Meta-Llama-3-8B-Instruct

Dieses Notebook zeigt die Verwendung des vortrainierten Sprachmodells Meta-Llama-3-8B-Instruct zur Generierung von Gegenreden für einen Datensatz mit 100 verschwörungstheoretischen Kommentaren. Das Skript ist für die Ausführung auf einer GPU optimiert, um die hohen Rechenanforderungen effizient zu bewältigen und die Generierung von Gegenreden reibungslos zu ermöglichen. Die erzeugten Antworten folgen spezifischen Richtlinien, die besonders Wert auf Empathie, Klarheit und den richtigen Ton legen. Das Endergebnis ist ein neuer Datensatz, der sowohl die ursprünglichen Verschwörungstheorie-Kommentare als auch die entsprechenden Gegenreden enthält und für weiterführende Analysen oder Trainingszwecke genutzt werden kann.

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load dataset with conspiracy theory comments
ct_dataset = pd.read_csv('../data/qanon_deepstate_comments.csv')

# Initialize model and tokenizer 
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
).to(device)

# Function to generate counter speech
def generate_counter_speech(comment):
    messages = [
        {"role": "system", 
         "content": """You are a trained expert in generating counter speech to conspiracy theory comments. 
                        Follow these response guidelines: 
                        1. Show empathy and positivity in your response.
                        2. Do not state 'this is a conspiracy theory' directly.
                        3. Use narrative storytelling, including a first-person perspective, detailed accounts of characters' internal lives, metaphors and figurative language.  Include a relatable protagonist (well-known figures only) or credible real-life examples to illustrate your point.
                        4. Ensure clarity in your argumentation with defined objectives.
                        5. Challenge the statement and refute it with specific facts from reliable sources. If appropriate, ask for sources or factual basis.
                        6. Maintain a respectful and calm tone throughout your response. Be cautious with sarcasm, humor, parody, and satire.
                        7. Always respond concisely, directly, and clearly. Limit your response to 800 characters.
                        """},
        {"role": "user", "content": f"Generate counter speech to the following conspiracy theory comment: {comment}"}
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(device)

    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
    
    outputs = model.generate(
        input_ids,
        max_new_tokens=350,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    response = outputs[0][input_ids.shape[-1]:]
    return tokenizer.decode(response, skip_special_tokens=True)

# Generate counter speech for every comment and save in appropriate column
ct_dataset['counter_speech_llama3'] = ct_dataset['comment_text'].apply(generate_counter_speech)

# Save results in a new file
ct_dataset.to_csv('../data/counterspeech_dataset_llama3.csv', index=False)

# Clean up resources to prevent memory leaks
del model, tokenizer
torch.cuda.empty_cache()